# Object Relational Tutorial

source: https://docs.sqlalchemy.org/en/13/orm/tutorial.html

The SQLAlchemy Object Relational Mapper presents a method of associating:
1. user-defined Python classes with database tables, and
2. instances of those classes (objects) with rows in their corresponding tables.

It includes:
1. a system that transparently synchronizes all changes in state between objects and their related rows, called a [unit of work](https://docs.sqlalchemy.org/en/13/glossary.html#term-unit-of-work), as well as
2. a system for expressing database queries in terms of the user defined classes and their defined relationships between each other.

In [1]:
import sqlalchemy
sqlalchemy.__version__

'1.3.19'

In [2]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:', echo=True)

type(engine)

sqlalchemy.engine.base.Engine

When using the ORM, we typically don’t use the `Engine` directly once created; instead, it’s used behind the scenes by the ORM as we’ll see shortly.

### Declare a Mapping

When using the ORM, the configurational process:
- starts by describing the database tables we’ll be dealing with, and
- then by defining our own classes which will be mapped to those tables.

In modern SQLAlchemy, these two tasks are usually performed together, using a system known as `Declarative`, which allows us to create classes that include directives to describe the actual database table they will be mapped to.

Classes mapped using the `Declarative` system are defined in terms of a base class which maintains a catalog of classes and tables relative to that base - this is known as the **declarative base class**.

In [3]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [4]:
from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    
    def __repr__(self):
        return (
            f"<User(name={self.name}, fullname={self.fullname},"
            f" nickname={self.nickname})>"
        )

When our class is constructed, Declarative replaces all the `Column` objects with special Python accessors known as "descriptors"; this is a process known as "instrumentation".

The “instrumented” mapped class will provide us with the means to refer to our table in a SQL context as well as to persist and load the values of columns from the database.

### Create a Schema and an Instance of the Mapped Class

In [5]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

In [6]:
type(User.__table__)

sqlalchemy.sql.schema.Table

In [7]:
# issue CREATE TABLE statements to the database
# for all tables that don’t yet exist

Base.metadata.create_all(engine)

2020-10-15 20:36:43,935 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-10-15 20:36:43,943 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 20:36:43,951 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-10-15 20:36:43,957 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 20:36:43,964 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2020-10-15 20:36:43,967 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 20:36:43,981 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2020-10-15 20:36:43,994 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 20:36:44,002 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2020-10-15 20:36:44,006 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 20:36:44,010 INFO sqlalchemy.engine.base.Engine COMMIT


In [8]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')

ed_user

<User(name=ed, fullname=Ed Jones, nickname=edsnickname)>

In [9]:
str(ed_user.id)

'None'

### Creating a Session

We’re now ready to start talking to the database. The ORM’s “handle” to the database is the `Session`.

In the most general sense, the `Session` establishes all conversations with the database and represents a “holding zone” for all the objects which you’ve loaded or associated with it during its lifespan.

In [10]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [11]:
session = Session()

In [12]:
session.add(ed_user)

At this point, we say that the instance is **pending**; no SQL has yet been issued and the object is not yet represented by a row in the database.

The `Session` will issue the SQL to persist `ed_user` as soon as is needed; this process is known as a **flush**. For example, if we query the database:
- first all pending information will be flushed, and
- immediately thereafter the query will be issued.

In [13]:
query = session.query(User).filter_by(name='ed')

print(type(query))
print()

our_user = query.first()

<class 'sqlalchemy.orm.query.Query'>

2020-10-15 20:36:44,182 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-15 20:36:44,187 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-10-15 20:36:44,190 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edsnickname')
2020-10-15 20:36:44,194 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2020-10-15 20:36:44,199 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


In [14]:
# Note: the Session has identified that
# the row returned is **the same row**
# as one already represented within its internal map of objects,
# so we actually got back the identical instance
# as that which we just added:

# Note: executing the `Query` returned the very same instance
# as the one that was added to the `Session`:
our_user is ed_user

True

[The `Session`] provides the entrypoint to acquire a `Query` object, which sends queries to the database (using the `Session` object’s current database connection).

Doing so populates result rows into objects that are then stored in the `Session`, inside a structure called the "Identity Map" - a data structure that maintains unique copies of each object, where “unique” means “only one object with a particular primary key”.

An Identity Map is a mapping that’s associated with an ORM `Session` object, and maintains a mapping/correspondence between Python objects and their database identities. (Thus, an Identity Map keeps a record of all objects that have been read from the database in a single business transaction.)

In [15]:
# In addition,
# if we look at Ed's `id` attribute, which earlier was `None`,
# it now has a value:
ed_user.id, our_user.id

(1, 1)

In [16]:
# We can add more User objects at once:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', nickname='windy'),
    User(name='mary', fullname='Mary Contrary', nickname='mary'),
    User(name='fred', fullname='Fred Flintstone', nickname='freddy'),
])

# Also, let's change Ed's nickname:
ed_user.nickname = 'eddie'

In [17]:
# The `Session` is paying attention...
# exhibit A:
session.dirty

IdentitySet([<User(name=ed, fullname=Ed Jones, nickname=eddie)>])

In [18]:
# exhibit B:
session.new

IdentitySet([<User(name=wendy, fullname=Wendy Williams, nickname=windy)>, <User(name=mary, fullname=Mary Contrary, nickname=mary)>, <User(name=fred, fullname=Fred Flintstone, nickname=freddy)>])

In [19]:
# issue all remaining changes to the database and commit the transaction,
# which has been in progress throughout
session.commit()

2020-10-15 20:36:44,343 INFO sqlalchemy.engine.base.Engine UPDATE users SET nickname=? WHERE users.id = ?
2020-10-15 20:36:44,346 INFO sqlalchemy.engine.base.Engine ('eddie', 1)
2020-10-15 20:36:44,349 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-10-15 20:36:44,353 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'windy')
2020-10-15 20:36:44,356 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-10-15 20:36:44,358 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'mary')
2020-10-15 20:36:44,360 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-10-15 20:36:44,362 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flintstone', 'freddy')
2020-10-15 20:36:44,364 INFO sqlalchemy.engine.base.Engine COMMIT


### Rolling back

Since the `Session` works within a transaction, we can roll back changes made too.

In [20]:
# Make 2 changes that we'll revert

ed_user.name = 'Edwardo'

fake_user = User(name='fakeuser', fullname='Invalid', nickname='12345')
session.add(fake_user)

In [21]:
# Querying the session shows that
# both changes are flushed into the current transaction
session.query(User).filter(
    User.name.in_(['Edwardo', 'fakeuser'])
).all()

2020-10-15 20:36:44,395 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-15 20:36:44,399 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2020-10-15 20:36:44,402 INFO sqlalchemy.engine.base.Engine (1,)
2020-10-15 20:36:44,404 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.id = ?
2020-10-15 20:36:44,406 INFO sqlalchemy.engine.base.Engine ('Edwardo', 1)
2020-10-15 20:36:44,415 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-10-15 20:36:44,418 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', '12345')
2020-10-15 20:36:44,421 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2020-10-15 20:36:44,422 INFO sqlalchemy.engine.base.Engine ('Ed

[<User(name=Edwardo, fullname=Ed Jones, nickname=eddie)>,
 <User(name=fakeuser, fullname=Invalid, nickname=12345)>]

In [22]:
# Roll back the changes, and verify that has worked as expected.

session.rollback()

print(ed_user.name == 'ed')
print(fake_user in session)

print()
print(
    session.query(User).filter(
        User.name.in_(['ed', 'fakeuser'])
    ).all()
)

2020-10-15 20:36:44,460 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-10-15 20:36:44,465 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-15 20:36:44,468 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2020-10-15 20:36:44,470 INFO sqlalchemy.engine.base.Engine (1,)
True
False

2020-10-15 20:36:44,476 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2020-10-15 20:36:44,484 INFO sqlalchemy.engine.base.Engine ('ed', 'fakeuser')
[<User(name=ed, fullname=Ed Jones, nickname=eddie)>]


### Querying

A `Query` object is created using the `query()` method on `Session`.

This function takes a variable number of arguments, which can be any combination of (a) classes and (b) class-instrumented descriptors.

In [23]:
for instance in session.query(User).order_by(User.id):
    print(type(instance), instance.name, instance.fullname)

2020-10-15 20:36:44,571 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
2020-10-15 20:36:44,581 INFO sqlalchemy.engine.base.Engine ()
<class '__main__.User'> ed Ed Jones
<class '__main__.User'> wendy Wendy Williams
<class '__main__.User'> mary Mary Contrary
<class '__main__.User'> fred Fred Flintstone


In [24]:
for named_tuple in session.query(User.name, User).all():
    print(f"{named_tuple.name}   <---   {named_tuple.User}")

2020-10-15 20:36:44,618 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, users.id AS users_id, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users
2020-10-15 20:36:44,622 INFO sqlalchemy.engine.base.Engine ()
ed   <---   <User(name=ed, fullname=Ed Jones, nickname=eddie)>
wendy   <---   <User(name=wendy, fullname=Wendy Williams, nickname=windy)>
mary   <---   <User(name=mary, fullname=Mary Contrary, nickname=mary)>
fred   <---   <User(name=fred, fullname=Fred Flintstone, nickname=freddy)>


#### Controlling names of columns and entities

- You can control the names of individual column expressions using the `ColumnElement.label()` construct
- The name given to a full entity such as `User` ... can be controlled using `aliased()`

[no examples]

#### Basic operations with `Query`

In [25]:
# LIMIT and OFFSET
for u in session.query(User).order_by(User.id)[1:3]:
    print(u.id, u)

2020-10-15 20:36:44,647 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?
2020-10-15 20:36:44,651 INFO sqlalchemy.engine.base.Engine (2, 1)
2 <User(name=wendy, fullname=Wendy Williams, nickname=windy)>
3 <User(name=mary, fullname=Mary Contrary, nickname=mary)>


In [26]:
# filtering results
# (using keyword args)
for u in session.query(User).filter_by(fullname="Ed Jones"):
    print(u.id, u)

2020-10-15 20:36:44,713 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.fullname = ?
2020-10-15 20:36:44,724 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)
1 <User(name=ed, fullname=Ed Jones, nickname=eddie)>


In [27]:
# filtering results
# (using more flexible SQL expression language constructs)
for u in session.query(User).filter(User.fullname == "Wendy Williams"):
    print(u.id, u)

2020-10-15 20:36:44,762 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.fullname = ?
2020-10-15 20:36:44,764 INFO sqlalchemy.engine.base.Engine ('Wendy Williams',)
2 <User(name=wendy, fullname=Wendy Williams, nickname=windy)>


The `Query` object is fully **generative**, meaning that most method calls return a new `Query` object upon which further criteria may be added.

In [28]:
for u in session.query(User).filter(
    User.name == "mary"
).filter(
    User.fullname == "Mary Contrary"
):
    print(u.id, u)

2020-10-15 20:36:44,804 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2020-10-15 20:36:44,818 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary')
3 <User(name=mary, fullname=Mary Contrary, nickname=mary)>


A number of methods on Query immediately issue SQL and return a value containing loaded database results. Here’s a brief tour:
- `Query.all()` returns a list
- `Query.first()` applies a LIMIT of 1 (and an OFFSET of 0) and returns the first result as a scalar
- `Query.one()` fully fetches all rows, and if not exactly one object identity or composite row is present in the result, raises an error.
- `Query.one_or_none()` is like `Query.one()`, except that if no results are found, it doesn’t raise an error; it just returns `None`. Like `Query.one()`, however, it does raise an error if multiple results are found.
- `Query.scalar()` invokes the `Query.one()` method, and upon success returns the first column of the row

#### Using Textual SQL

Literal strings can be used flexibly with `Query`, by specifying their use with the `text()` [SQL Expression Language] construct, which is accepted by most applicable methods.

Nice examples can be found at https://docs.sqlalchemy.org/en/13/orm/tutorial.html#using-textual-sql

#### Counting

In [29]:
session.query(User).filter(
    User.name.ilike("%ed")
).count()

2020-10-15 20:36:44,859 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE lower(users.name) LIKE lower(?)) AS anon_1
2020-10-15 20:36:44,866 INFO sqlalchemy.engine.base.Engine ('%ed',)


2

In [30]:
# For situations where the “thing to be counted” needs to be indicated specifically:

from sqlalchemy import func

session.query(
    User.name,
    func.count(User.name),
).group_by(
    User.name
).all()

2020-10-15 20:36:44,887 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, count(users.name) AS count_1 
FROM users GROUP BY users.name
2020-10-15 20:36:44,893 INFO sqlalchemy.engine.base.Engine ()


[('ed', 1), ('fred', 1), ('mary', 1), ('wendy', 1)]

In [31]:
# To achieve our simple
#     `SELECT count(*) FROM table`:

session.query(
    func.count("*")
).select_from(
    User
).scalar()

# # or:
# session.query(
#     func.count(User.id)
# ).scalar()

2020-10-15 20:36:44,938 INFO sqlalchemy.engine.base.Engine SELECT count(?) AS count_1 
FROM users
2020-10-15 20:36:44,942 INFO sqlalchemy.engine.base.Engine ('*',)


4

### Building a Relationship

Let’s consider how a second table, related to `User`, can be mapped and queried.

Let us suppose that users in our system can store any number of email addresses associated with their username. This implies a basic one-to-many association from the `users` to a new table which stores email addresses, which we will call `addresses`.

Using `declarative`, we define this table along with its mapped class, `Address`:

In [32]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Address(Base):
    __tablename__ = "addresses"
    
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    
    # the `ForeignKey` construct ... is a directive applied to `Column`,
    # which indicates that values in this column should be constrained
    # to be values present in the named "remote" column
    user_id = Column(Integer, ForeignKey("users.id"))
    # the `relationship()` directive (a) tells the ORM that the `Address`
    # class itself should be linked to the `User` class (using the
    # `Address.user` attribute), and (b) uses the foreign-key directive
    # to determine the nature of this linkage (i.e. that `Address.user`
    # will be many-to-one), and (c) assigns the
    # `relationship.back_populates` parameter to refer to the
    # complementary attribute name
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return (
            f"<Address email_address={self.email_address}>"
        )

User.addresses = relationship(
    "Address",
    order_by=Address.id,
    back_populates="user"
)

#### Did you know?

- FOREIGN KEY columns can automatically update themselves, in response to a change in the referenced column or row. This is known as the CASCADE *referential action*, and is a built-in function of the relational database.
- more "Did you know?" facts on https://docs.sqlalchemy.org/en/13/orm/tutorial.html#building-a-relationship

We’ll need to create the `addresses` table in the database, so we will issue another CREATE from our metadata, which will skip over tables which have already been created:

In [33]:
Base.metadata.create_all(engine)

2020-10-15 20:36:45,022 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2020-10-15 20:36:45,025 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 20:36:45,028 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("addresses")
2020-10-15 20:36:45,031 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 20:36:45,034 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("addresses")
2020-10-15 20:36:45,036 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 20:36:45,039 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	email_address VARCHAR NOT NULL, 
	user_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2020-10-15 20:36:45,045 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 20:36:45,047 INFO sqlalchemy.engine.base.Engine COMMIT


### Working with Related Objects

In [34]:
jack = User(
    name="jack",
    fullname="Jack Bean",
    nickname="jacks-random-nickname",
)
jack.addresses

[]

In [35]:
jack.addresses = [
    Address(email_address="jack@google.com"),
    Address(email_address="j25@yahoo.com"),
]

In [36]:
# add and commit `jack` to the database,
# which - thanks to a process known as CASCADING -
# is also going to add the two `Address` members
# in the corresponding `addresses` collection
session.add(jack)
session.commit()

2020-10-15 20:36:45,098 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-10-15 20:36:45,105 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Bean', 'jacks-random-nickname')
2020-10-15 20:36:45,110 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2020-10-15 20:36:45,115 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 5)
2020-10-15 20:36:45,117 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2020-10-15 20:36:45,119 INFO sqlalchemy.engine.base.Engine ('j25@yahoo.com', 5)
2020-10-15 20:36:45,124 INFO sqlalchemy.engine.base.Engine COMMIT


In [37]:
jack = session.query(User).filter_by(
    name="jack"
).one()

jack

2020-10-15 20:36:45,142 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-15 20:36:45,148 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2020-10-15 20:36:45,150 INFO sqlalchemy.engine.base.Engine ('jack',)


<User(name=jack, fullname=Jack Bean, nickname=jacks-random-nickname)>

In [38]:
jack.addresses

2020-10-15 20:36:45,183 INFO sqlalchemy.engine.base.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE ? = addresses.user_id ORDER BY addresses.id
2020-10-15 20:36:45,189 INFO sqlalchemy.engine.base.Engine (5,)


[<Address email_address=jack@google.com>,
 <Address email_address=j25@yahoo.com>]

### Querying with Joins

http://en.wikipedia.org/wiki/Join_%28SQL%29 offers a good introduction to JOIN techniques, several of which we’ll illustrate here.

In [39]:
# construct a simple implicit JOIN
for u, a in session.query(
    User, Address
).filter(
    User.id == Address.user_id
).filter(
    Address.email_address == "jack@google.com"
).all():
    print(u)
    print(a)

2020-10-15 20:36:45,217 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM users, addresses 
WHERE users.id = addresses.user_id AND addresses.email_address = ?
2020-10-15 20:36:45,219 INFO sqlalchemy.engine.base.Engine ('jack@google.com',)
<User(name=jack, fullname=Jack Bean, nickname=jacks-random-nickname)>
<Address email_address=jack@google.com>


In [40]:
# The actual JOIN syntax
session.query(User).join(
    Address
).filter(
    Address.email_address == "jack@google.com"
).all()

2020-10-15 20:36:45,241 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users JOIN addresses ON users.id = addresses.user_id 
WHERE addresses.email_address = ?
2020-10-15 20:36:45,245 INFO sqlalchemy.engine.base.Engine ('jack@google.com',)


[<User(name=jack, fullname=Jack Bean, nickname=jacks-random-nickname)>]

In the last example, `Query.join()` knew how to join between `User` and `Address` because there was only one foreign key between them.

If there are no foreign keys, or several, `Query.join()` works better when one of the following forms is used:
```
query.join(Address, User.id==Address.user_id)    # explicit condition
query.join(User.addresses)                       # specify relationship from left to right
query.join(Address, User.addresses)              # same, with explicit target
```

#### Using Aliases

When querying across multiple tables, if the same table needs to be referenced more than once, SQL typically requires that the table be aliased with another name, so that it can be distinguished against other occurrences of that table.

In the SQLAlchemy ORM, aliasing a table with another name can be accomplished by means of the `aliased()` construct. There are 2 equivalent ways of making the target of a relationship JOIN refer to an `aliased()` object. As a concrete example, if the aliased object is `adalias1 = aliased(Address)`, then the 2 ways are as follows:
```
q = query.join(User.addresses.of_type(adalias1))
q = query.join(adalias1, User.addresses)
```

#### Using Subqueries

Suppose we wanted to load `User` objects along with a count of how many `Address` records each user has.

The best way to generate SQL like this is:
```
SELECT users.*, adr_count.address_count
FROM users
LEFT JOIN (
    SELECT user_id, count(*) AS address_count
    FROM addresses GROUP BY user_id
) AS adr_count
ON users.id = adr_count.user_id;
```

Using the `Query`, we build a statement like that from the inside out.

In [41]:
# the `subquery()` method on `Query` produces a SQL expression construct
# representing a SELECT statement embedded within an alias
stmt = session.query(
    Address.user_id,
    func.count("*").label("address_count"),
).group_by(
    Address.user_id
).subquery()

type(stmt)

sqlalchemy.sql.selectable.Alias

In [42]:
for u, count in session.query(
    User, stmt.c.address_count
).outerjoin(
    stmt, User.id == stmt.c.user_id
).order_by(
    User.id
):
    print(u, count)

2020-10-15 20:36:45,392 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, anon_1.address_count AS anon_1_address_count 
FROM users LEFT OUTER JOIN (SELECT addresses.user_id AS user_id, count(?) AS address_count 
FROM addresses GROUP BY addresses.user_id) AS anon_1 ON users.id = anon_1.user_id ORDER BY users.id
2020-10-15 20:36:45,398 INFO sqlalchemy.engine.base.Engine ('*',)
<User(name=ed, fullname=Ed Jones, nickname=eddie)> None
<User(name=wendy, fullname=Wendy Williams, nickname=windy)> None
<User(name=mary, fullname=Mary Contrary, nickname=mary)> None
<User(name=fred, fullname=Fred Flintstone, nickname=freddy)> None
<User(name=jack, fullname=Jack Bean, nickname=jacks-random-nickname)> 2


#### Selecting Entities from Subqueries

Above, we just selected a result that included a column from a subquery. 

What if we wanted our subquery to map to an entity? For this we associate an "alias" of a mapped class to a subquery:
```
stmt = session.query(
    Address
).filter(
    Address.email_address != "j25@yahoo.com"
).subquery()

adalias = aliased(Address, alias=stmt)

for user, address in session.query(
    User, adalias
).join(
    adalias, User.addresses
):
    print(user)
    print(address)
```

#### Using EXISTS

The EXISTS keyword in SQL is a boolean operator which returns True if the given expression contains any rows.

The EXISTS keyword/operator:
- may be used in many scenarios in place of JOINs, and
- is also useful for locating rows which do not have a corresponding row in a related table.

An example of the latter query is:
```
SELECT users.name
FROM users
WHERE EXISTS (
    SELECT *
    FROM addresses
    WHERE addresses.user_id = users.id
);
```

In [43]:
# There is an explicit EXISTS construct, which looks like this:

from sqlalchemy.sql import exists

stmt = exists().where(
    Address.user_id == User.id
)

for named_tuple in session.query(
    User.name
).filter(
    exists().where(
        Address.user_id == User.id
    )
):
    print(named_tuple)

2020-10-15 20:36:45,428 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE EXISTS (SELECT * 
FROM addresses 
WHERE addresses.user_id = users.id)
2020-10-15 20:36:45,433 INFO sqlalchemy.engine.base.Engine ()
('jack',)


In [44]:
# (Essentially) The same statement can be expressed
# along the `User.addresses` relationship using `Comparator.any()`,
# which is a `Query` operator that makes use of EXISTS automatically.

for named_tuple in session.query(
    User.name
).filter(
    User.addresses.any()
):
    print(named_tuple)

2020-10-15 20:36:45,453 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE EXISTS (SELECT 1 
FROM addresses 
WHERE users.id = addresses.user_id)
2020-10-15 20:36:45,461 INFO sqlalchemy.engine.base.Engine ()
('jack',)


In [45]:
# `Comparator.any()` can limit the rows matched, by taking a criterion:
for named_tuple in session.query(
    User.name
).filter(
    User.addresses.any(
        Address.email_address.like("%google%")
    )
):
    print(named_tuple)

2020-10-15 20:36:45,488 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE EXISTS (SELECT 1 
FROM addresses 
WHERE users.id = addresses.user_id AND addresses.email_address LIKE ?)
2020-10-15 20:36:45,492 INFO sqlalchemy.engine.base.Engine ('%google%',)
('jack',)


In [46]:
# While the `Comparator.any()` operator is for one-to-many relationships,
# its equivalent for many-to-one relationships is `Comparator.has()`
session.query(
    Address
).filter(
    ~Address.user.has(
        User.name == 'jack'
    )
).all()

2020-10-15 20:36:45,520 INFO sqlalchemy.engine.base.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE NOT (EXISTS (SELECT 1 
FROM users 
WHERE users.id = addresses.user_id AND users.name = ?))
2020-10-15 20:36:45,523 INFO sqlalchemy.engine.base.Engine ('jack',)


[]

### Deleting

Let’s try to delete `jack` and see how that goes.

In [47]:
session.delete(jack)

session.query(
    User
).filter_by(
    name="jack"
).count()

2020-10-15 20:36:45,555 INFO sqlalchemy.engine.base.Engine UPDATE addresses SET user_id=? WHERE addresses.id = ?
2020-10-15 20:36:45,558 INFO sqlalchemy.engine.base.Engine ((None, 1), (None, 2))
2020-10-15 20:36:45,561 INFO sqlalchemy.engine.base.Engine DELETE FROM users WHERE users.id = ?
2020-10-15 20:36:45,563 INFO sqlalchemy.engine.base.Engine (5,)
2020-10-15 20:36:45,565 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?) AS anon_1
2020-10-15 20:36:45,567 INFO sqlalchemy.engine.base.Engine ('jack',)


0

So far, so good. How about Jack’s `Address` objects?

In [48]:
session.query(
    Address
).filter(
    Address.email_address.in_([
        "jack@google.com",
        "j25@yahoo.com",
    ])
).count()

2020-10-15 20:36:45,594 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE addresses.email_address IN (?, ?)) AS anon_1
2020-10-15 20:36:45,599 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 'j25@yahoo.com')


2

Uh oh, they’re still there ! Analyzing the first flush SQL, we can see that the `user_id` column of each address was set to NULL, but the rows weren’t deleted.

SQLAlchemy doesn’t assume a "delete cascade". That has be told to it explicitly.

#### Configuring delete/delete-orphan Cascade

We will configure **cascade** options on the `User.addresses` relationship to change the behavior.

While SQLAlchemy allows you to add new attributes and relationships to mappings at any point in time, in this case the existing relationship needs to be removed, so we need to tear down the mappings completely and start again:

In [49]:
session.close()

2020-10-15 20:36:45,621 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [50]:
# use a new declarative_base()
Base = declarative_base()

In [51]:
# declare the `User` class,
# adding in the `addresses` relationship including the cascade information

class User(Base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    
    addresses = relationship(
        "Address",
        back_populates="user",
        cascade="all, delete, delete-orphan",
    )
    
    def __repr__(self):
        return (
            f"<User(name={self.name}, fullname={self.fullname},"
            f" nickname={self.nickname})>"
        )

In [52]:
class Address(Base):
    __tablename__ = "addresses"
    
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    
    user_id = Column(Integer, ForeignKey("users.id"))
    user = relationship(
        "User",
        back_populates="addresses",
    )
    
    def __repr__(self):
        return (
            f"<Address email_address={self.email_address}>"
        )

In [53]:
# load Jack by primary key
jack = session.query(User).get(5)

2020-10-15 20:36:45,706 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-15 20:36:45,716 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2020-10-15 20:36:45,718 INFO sqlalchemy.engine.base.Engine (5,)


In [54]:
# remove one Address (lazy load fires off)
del jack.addresses[1]

2020-10-15 20:36:45,737 INFO sqlalchemy.engine.base.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE ? = addresses.user_id
2020-10-15 20:36:45,740 INFO sqlalchemy.engine.base.Engine (5,)


In [55]:
# verify that only one Address remains
session.query(
    Address
).filter(
    Address.email_address.in_([
        "jack@google.com",
        "j25@yahoo.com",
])).count()

2020-10-15 20:36:45,759 INFO sqlalchemy.engine.base.Engine DELETE FROM addresses WHERE addresses.id = ?
2020-10-15 20:36:45,764 INFO sqlalchemy.engine.base.Engine (2,)
2020-10-15 20:36:45,770 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE addresses.email_address IN (?, ?)) AS anon_1
2020-10-15 20:36:45,773 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 'j25@yahoo.com')


1

In [56]:
# deleting `jack` will delete
# both `jack` and the remaining `Address` associated with the user
session.delete(jack)

In [57]:
session.query(
    User
).filter_by(
    name="jack"
).count()

2020-10-15 20:36:45,828 INFO sqlalchemy.engine.base.Engine DELETE FROM addresses WHERE addresses.id = ?
2020-10-15 20:36:45,834 INFO sqlalchemy.engine.base.Engine (1,)
2020-10-15 20:36:45,839 INFO sqlalchemy.engine.base.Engine DELETE FROM users WHERE users.id = ?
2020-10-15 20:36:45,841 INFO sqlalchemy.engine.base.Engine (5,)
2020-10-15 20:36:45,844 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?) AS anon_1
2020-10-15 20:36:45,846 INFO sqlalchemy.engine.base.Engine ('jack',)


0

In [58]:
session.query(
    Address
).filter(
    Address.email_address.in_([
        "jack@google.com",
        "j25@yahoo.com",
])).count()

2020-10-15 20:36:45,879 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE addresses.email_address IN (?, ?)) AS anon_1
2020-10-15 20:36:45,882 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 'j25@yahoo.com')


0

### Building a Many To Many Relationship

Let us suppose our application needs to allows `User`s to write `BlogPost`s, which have `Keyword`s associated with them.

For a plain many-to-many relationship, we need to create an un-mapped `Table` construct to serve as "an association table".

In what follows, the association table is defined so as to only contain columns which reference the two sides of the relationship. (If it has **any other columns** [such as its own primary key, or foreign keys to other tables], SQLAlchemy **requires a different usage pattern** called the “association object”, described at https://docs.sqlalchemy.org/en/13/orm/basic_relationships.html#association-pattern.)

In [59]:
from sqlalchemy import Table

# The metadata is used as a point of association (of `post_keywords`)
# with other tables referenced via foreign key.
post_keywords = Table(
    "post_keywords",
    Base.metadata,
    Column("post_id", ForeignKey("posts.id"), primary_key=True),
    Column("keyword_id", ForeignKey("keywords.id"), primary_key=True),
)

type(post_keywords)

sqlalchemy.sql.schema.Table

In [60]:
from sqlalchemy import Text


class BlogPost(Base):
    __tablename__ = "posts"
    
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey("users.id"))
    headline = Column(String(255), nullable=False)
    body = Column(Text)
    
    # many-to-many BlogPost<->Keyword
    keywords = relationship(
        "Keyword",
        secondary=post_keywords,  # references a `Table` object representing the association table
        back_populates="posts",
    )
    
    def __init__(self, headline, body, author):
        self.headline = headline
        self.body = body
        self.author = author
    
    def __repr__(self):
        return (
            f"BlogPost({self.headline}, {self.body}, {self.author})"
        )


class Keyword(Base):
    __tablename__ = "keywords"
    
    id = Column(Integer, primary_key=True)
    keyword = Column(String(50), nullable=False, unique=True)
    
    posts = relationship(
        "BlogPost",
        secondary=post_keywords,    # references a `Table` object representing the association table
        back_populates="keywords",
    )
    
    def __init__(self, keyword):
        self.keyword = keyword

In [61]:
# We would also like our `BlogPost` class to have an `author` field.
# We will add this as another bidirectional relationship,
# except one issue we’ll have is that
# a single user might have lots of blog posts.

BlogPost.author = relationship(
    User,
    back_populates="posts",
)

User.posts = relationship(
    BlogPost,
    back_populates="author",
    lazy="dynamic",  # enables filtering of the results before loading them
)

In [62]:
# Create new tables:

Base.metadata.create_all(engine)

2020-10-15 20:36:46,027 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2020-10-15 20:36:46,030 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 20:36:46,033 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("addresses")
2020-10-15 20:36:46,035 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 20:36:46,040 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("post_keywords")
2020-10-15 20:36:46,042 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 20:36:46,044 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("post_keywords")
2020-10-15 20:36:46,046 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 20:36:46,049 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("posts")
2020-10-15 20:36:46,053 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 20:36:46,057 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("posts")
2020-10-15 20:36:46,059 INFO sqlalchemy.engine.base.Engine ()
2020-10-15 20:36:46,067 INFO sqlalchemy.engine.base.Engine PRA

In [63]:
# give Wendy some blog posts
wendy = session.query(
    User
).filter_by(
    name="wendy"
).one()

2020-10-15 20:36:46,155 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2020-10-15 20:36:46,159 INFO sqlalchemy.engine.base.Engine ('wendy',)


In [64]:
print(type(wendy.posts))

<class 'sqlalchemy.orm.dynamic.AppenderQuery'>


In [65]:
post = BlogPost(
    "Wendy's Blog Post",
    "This is a test",
    wendy,
)

session.add(post)

In [66]:
post.keywords.append(
    Keyword("wendy")
)

post.keywords.append(
    Keyword("firstpost")
)

In [67]:
session.query(
    BlogPost
).filter(
    BlogPost.keywords.any(
        keyword="firstpost"
    )
).all()

2020-10-15 20:36:46,230 INFO sqlalchemy.engine.base.Engine INSERT INTO keywords (keyword) VALUES (?)
2020-10-15 20:36:46,234 INFO sqlalchemy.engine.base.Engine ('wendy',)
2020-10-15 20:36:46,237 INFO sqlalchemy.engine.base.Engine INSERT INTO keywords (keyword) VALUES (?)
2020-10-15 20:36:46,239 INFO sqlalchemy.engine.base.Engine ('firstpost',)
2020-10-15 20:36:46,242 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (user_id, headline, body) VALUES (?, ?, ?)
2020-10-15 20:36:46,244 INFO sqlalchemy.engine.base.Engine (2, "Wendy's Blog Post", 'This is a test')
2020-10-15 20:36:46,248 INFO sqlalchemy.engine.base.Engine INSERT INTO post_keywords (post_id, keyword_id) VALUES (?, ?)
2020-10-15 20:36:46,250 INFO sqlalchemy.engine.base.Engine ((1, 1), (1, 2))
2020-10-15 20:36:46,255 INFO sqlalchemy.engine.base.Engine SELECT posts.id AS posts_id, posts.user_id AS posts_user_id, posts.headline AS posts_headline, posts.body AS posts_body 
FROM posts 
WHERE EXISTS (SELECT 1 
FROM post_keywords,

[BlogPost(Wendy's Blog Post, This is a test, <User(name=wendy, fullname=Wendy Williams, nickname=windy)>)]

In [68]:
session.query(
    BlogPost
).filter(
    BlogPost.author == wendy
).filter(
    BlogPost.keywords.any(
        keyword="firstpost"
    )
).all()

2020-10-15 20:36:46,350 INFO sqlalchemy.engine.base.Engine SELECT posts.id AS posts_id, posts.user_id AS posts_user_id, posts.headline AS posts_headline, posts.body AS posts_body 
FROM posts 
WHERE ? = posts.user_id AND (EXISTS (SELECT 1 
FROM post_keywords, keywords 
WHERE posts.id = post_keywords.post_id AND keywords.id = post_keywords.keyword_id AND keywords.keyword = ?))
2020-10-15 20:36:46,352 INFO sqlalchemy.engine.base.Engine (2, 'firstpost')


[BlogPost(Wendy's Blog Post, This is a test, <User(name=wendy, fullname=Wendy Williams, nickname=windy)>)]

In [69]:
type(wendy.posts)

sqlalchemy.orm.dynamic.AppenderQuery

In [70]:
wendy.posts.filter(
    BlogPost.keywords.any(
        keyword="firstpost"
    )
).all()

2020-10-15 20:36:46,391 INFO sqlalchemy.engine.base.Engine SELECT posts.id AS posts_id, posts.user_id AS posts_user_id, posts.headline AS posts_headline, posts.body AS posts_body 
FROM posts 
WHERE ? = posts.user_id AND (EXISTS (SELECT 1 
FROM post_keywords, keywords 
WHERE posts.id = post_keywords.post_id AND keywords.id = post_keywords.keyword_id AND keywords.keyword = ?))
2020-10-15 20:36:46,393 INFO sqlalchemy.engine.base.Engine (2, 'firstpost')


[BlogPost(Wendy's Blog Post, This is a test, <User(name=wendy, fullname=Wendy Williams, nickname=windy)>)]